In [20]:
import jax
import jax.numpy as jnp
import numpy as np
from typing import Callable, Tuple
import pandas as pd

jax.config.update("jax_enable_x64", True)

In [3]:
beta = 0.98
alpha = 0.3
delta = 0.075

k_star = ((1 / beta - 1 + delta) / alpha) ** (1 / (alpha - 1))
print(f"{k_star=:0.3f}")

k_star=5.138


# Extended Path Algorithm


In [4]:
def F(k, k_prime):
    return jnp.log(jnp.power(k, alpha) + (1 - delta) * k - k_prime)


dF_1 = jax.grad(F, argnums=0)
dF_2 = jax.grad(F, argnums=1)


def q(k, k_p, k_p2):
    return dF_2(k, k_p) + beta * dF_1(k_p, k_p2)


def Q(k0, phi):
    lower_tail = jnp.array([q(k0, phi[0], phi[1])])
    interior = jax.vmap(q)(phi[:-2], phi[1:-1], phi[2:])
    upper_tail = jnp.array([q(phi[-2], phi[-1], k_star)])

    return jnp.concatenate([lower_tail, interior, upper_tail])

In [5]:
k0 = 0.95 * k_star
phi = jnp.repeat(k_star, 200).astype(jnp.float64)


def newton_extended_path(k0, phi):
    tol = 1e-14
    iter = 0
    dist = jnp.linalg.norm(Q(k0, phi))

    while dist > tol:
        print(f"{iter=}: {dist=}")
        J = jax.jacrev(lambda x: Q(k0, x))(phi)

        phi = phi - (jnp.linalg.inv(J) @ Q(k0, phi))
        dist = jnp.linalg.norm(Q(k0, phi))

        iter += 1

    return phi


phi_k95 = newton_extended_path(0.95 * k_star, phi)
phi_k90 = newton_extended_path(0.90 * k_star, phi_k95)
phi_k80 = newton_extended_path(0.80 * k_star, phi_k90)
phi_k60 = newton_extended_path(0.60 * k_star, phi_k80)

iter=0: dist=Array(0.21327126, dtype=float64)
iter=1: dist=Array(0.03101912, dtype=float64)
iter=2: dist=Array(0.00082659, dtype=float64)
iter=3: dist=Array(6.12335192e-07, dtype=float64)
iter=4: dist=Array(3.37293575e-13, dtype=float64)
iter=0: dist=Array(0.2279395, dtype=float64)
iter=1: dist=Array(0.03400567, dtype=float64)
iter=2: dist=Array(0.00095784, dtype=float64)
iter=3: dist=Array(7.94493024e-07, dtype=float64)
iter=4: dist=Array(5.48087794e-13, dtype=float64)
iter=0: dist=Array(0.6902485, dtype=float64)
iter=1: dist=Array(0.19663463, dtype=float64)
iter=2: dist=Array(0.02482011, dtype=float64)
iter=3: dist=Array(0.0004807, dtype=float64)
iter=4: dist=Array(1.85948071e-07, dtype=float64)
iter=5: dist=Array(2.82930487e-14, dtype=float64)
iter=0: dist=Array(29.11101042, dtype=float64)
iter=1: dist=Array(14.33066245, dtype=float64)
iter=2: dist=Array(6.94009305, dtype=float64)
iter=3: dist=Array(3.24492233, dtype=float64)
iter=4: dist=Array(1.40107123, dtype=float64)
iter=5: dis

In [6]:
def speed_of_adjustment_extended_path(k0, phi):
    gap_closed = k0 + 0.99 * abs(k0 - k_star)
    return int(jnp.min(jnp.where(phi > gap_closed)[0]))


extended_path_results = {
    "k0=0.95*kstar": speed_of_adjustment_extended_path(0.95 * k_star, phi_k95),
    "k0=0.90*kstar": speed_of_adjustment_extended_path(0.90 * k_star, phi_k90),
    "k0=0.80*kstar": speed_of_adjustment_extended_path(0.80 * k_star, phi_k80),
    "k0=0.60*kstar": speed_of_adjustment_extended_path(0.60 * k_star, phi_k60),
}

In [7]:
extended_path_results

{'k0=0.95*kstar': 39,
 'k0=0.90*kstar': 39,
 'k0=0.80*kstar': 40,
 'k0=0.60*kstar': 40}

# Value Function Iteration

In [8]:
k_bar = 15
N = 10_000

eps = 1e-4
k_init = np.linspace(0, k_bar, N) + eps
k_init = k_init.reshape(N, 1)

k_prime = np.linspace(0, k_bar, N) + eps
k_prime = k_prime.reshape(1, N)

constants = {
    "k": k_init,
    "k_prime": k_prime,
}

In [9]:
def calculate_value_grid(
    k: np.ndarray,
    k_prime: np.ndarray,
    v_next: np.ndarray,
) -> np.ndarray:
    """Computes a grid of utility payoffs

    Parameters
    ----------
        k np.ndarray:
            N X 1 array of values to compute K_init over

        k_prime np.ndarray:
            1 X N array of values to compute K_prime over

    Returns
    -------
        N X N  array of computed values

    """
    c = k**alpha + (1 - delta) * k - k_prime
    U = np.full_like(c, -np.inf, dtype=float)

    # ensures k_prime <= k^alpha + (1-delta) * k
    feasibilty_mask = c > 0

    U[feasibilty_mask] = np.log(c[feasibilty_mask])
    v_next_col = np.asarray(v_next).reshape(1, -1)

    return U + beta * v_next_col


def bellman_operator(
    value_function: Callable, v_arr: np.ndarray, **kwargs
) -> Tuple[np.ndarray, np.ndarray]:
    """Computes a single iteration of the bellman operator.

    Parameters
    ----------
    value_function: callable
        value_function

    v_next_arr: np.ndarray
        (N,) array of previous maximum guess

    **kwargs
        keyword arguments to properly specify the value_function grid
    """
    v_mat = value_function(v_next=v_arr, **kwargs)

    argmax_v_mat = np.argmax(v_mat, axis=1)
    v_arr_next = np.max(v_mat, axis=1)

    return argmax_v_mat, v_arr_next


def sup_norm_dist(x: np.ndarray, y: np.ndarray) -> float:
    return np.max(np.abs(x - y))


def l2_norm(x: np.ndarray, y: np.ndarray) -> float:
    return np.sqrt(np.sum(np.power(x - y, 2)))


def infinite_horizon_value_function_iteration(
    convergence_eps: float, d: Callable[[np.ndarray, np.ndarray], float]
):
    """Solves the infinite horizon value-function iteration until convergence.

    Parameters
    ----------
    convergence_eps: float
        epsilon used to define convergence

    d: Callable[np.ndarray, np.ndarray] -> float
        distance function used to define convergence
    """

    # initialize guess
    v_max_array = np.full(N, 2)
    i = 1

    while True:
        argmax_v_mat, v_max_array_new = bellman_operator(
            calculate_value_grid, v_max_array, **constants
        )

        dist = d(v_max_array_new, v_max_array)
        print(f"Iteration {i}, Distance {dist}")

        v_max_array = v_max_array_new
        i += 1

        if dist <= convergence_eps:
            break

    return argmax_v_mat, v_max_array


def g(k, optimal_index):
    """Discretized policy function"""
    k_discrete_idx = np.argmin(np.abs(k_init - k))
    return k_prime.ravel()[optimal_index[k_discrete_idx]]


def simulate_infinite(k0, T_max, argmax_v_mat):
    """Solve the infinite horizon problem"""
    # The homeworks asks to go to 1e-10 tolerance, but I don't have time for that, results are stable enough here
    ks = np.zeros(T_max)
    cs = np.zeros(T_max - 1)

    ks[0] = k0

    for t in range(1, T_max):
        ks[t] = g(ks[t - 1], argmax_v_mat)
        cs[t - 1] = ks[t - 1] ** alpha + (1 - delta) * ks[t - 1] - ks[t]

    return ks, cs

In [10]:
argmax_v_mat, _ = infinite_horizon_value_function_iteration(
    convergence_eps=1e-3, d=l2_norm
)

Iteration 1, Distance 205.56806804587427
Iteration 2, Distance 98.28254686553211
Iteration 3, Distance 67.25497972121823
Iteration 4, Distance 50.5617319971315
Iteration 5, Distance 40.03695647549519
Iteration 6, Distance 32.82522932150634
Iteration 7, Distance 27.61945025809437
Iteration 8, Distance 23.724769915978253
Iteration 9, Distance 20.733740462698975
Iteration 10, Distance 18.38995518506923
Iteration 11, Distance 16.523226005761693
Iteration 12, Distance 15.015824374372267
Iteration 13, Distance 13.783618166629463
Iteration 14, Distance 12.764935416929537
Iteration 15, Distance 11.91366321897913
Iteration 16, Distance 11.19478849647875
Iteration 17, Distance 10.58140915320985
Iteration 18, Distance 10.052663512801706
Iteration 19, Distance 9.592252962702524
Iteration 20, Distance 9.187369058171512
Iteration 21, Distance 8.827890357948352
Iteration 22, Distance 8.505780648429873
Iteration 23, Distance 8.214630600645835
Iteration 24, Distance 7.949307143118487
Iteration 25, Dist

In [11]:
ks_vfi_95, _ = simulate_infinite(k0=0.95 * k_star, T_max=200, argmax_v_mat=argmax_v_mat)
ks_vfi_90, _ = simulate_infinite(k0=0.90 * k_star, T_max=200, argmax_v_mat=argmax_v_mat)
ks_vfi_80, _ = simulate_infinite(k0=0.80 * k_star, T_max=200, argmax_v_mat=argmax_v_mat)
ks_vfi_60, _ = simulate_infinite(k0=0.60 * k_star, T_max=200, argmax_v_mat=argmax_v_mat)

In [12]:
vfi_results = {
    "k0=0.95*kstar": speed_of_adjustment_extended_path(0.95 * k_star, ks_vfi_95),
    "k0=0.90*kstar": speed_of_adjustment_extended_path(0.90 * k_star, ks_vfi_90),
    "k0=0.80*kstar": speed_of_adjustment_extended_path(0.80 * k_star, ks_vfi_80),
    "k0=0.60*kstar": speed_of_adjustment_extended_path(0.60 * k_star, ks_vfi_60),
}

vfi_results

{'k0=0.95*kstar': 33,
 'k0=0.90*kstar': 38,
 'k0=0.80*kstar': 40,
 'k0=0.60*kstar': 41}

# Perturbation Method

In [15]:
def f(k):
    return jnp.power(k, alpha) + ((1 - delta) * k)


def u(k, k_prime):
    return jnp.log(f(k) - k_prime)


sigma_f = -jax.grad(jax.grad(f))(k_star) / jax.grad(f)(k_star)
sigma_u = -jax.grad(jax.grad(u))(k_star, k_star) / jax.grad(u)(k_star, k_star)

nu = 1 + (1 / beta) + (sigma_f / sigma_u)

_lambdas = jnp.roots(jnp.array([1, -nu, 1 / beta])).astype(jnp.float64)
_lambda = _lambdas[_lambdas < 1][0]
_lambda

Array(0.89270501, dtype=float64)

In [42]:
T = int(jnp.floor(jnp.log(0.01) / jnp.log(_lambda)) + 1)

In [44]:
perturbation_results = {
    "k0=0.95*kstar": T,
    "k0=0.90*kstar": T,    
    "k0=0.80*kstar": T,    
    "k0=0.60*kstar": T
}

perturbation_results

{'k0=0.95*kstar': 41,
 'k0=0.90*kstar': 41,
 'k0=0.80*kstar': 41,
 'k0=0.60*kstar': 41}

In [45]:
res = pd.DataFrame([extended_path_results, vfi_results, perturbation_results])

res.index = ["Extended Path", "Value Function Iteration", "Perturbation"]
res.columns = [
    "$k_0 = 0.95k^*$",
    "$k_0 = 0.90k^*$",
    "$k_0 = 0.80k^*$",
    "$k_0 = 0.60k^*$",
]
print(res.to_latex())

\begin{tabular}{lrrrr}
\toprule
 & $k_0 = 0.95k^*$ & $k_0 = 0.90k^*$ & $k_0 = 0.80k^*$ & $k_0 = 0.60k^*$ \\
\midrule
Extended Path & 39 & 39 & 40 & 40 \\
Value Function Iteration & 33 & 38 & 40 & 41 \\
Perturbation & 41 & 41 & 41 & 41 \\
\bottomrule
\end{tabular}

